In [1]:
# Example Code to Extract features from any layer of any model

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torchvision.models as models
from torch.autograd import Variable
import os

In [3]:
print("torch version {}".format(torch.__version__))
print("cuda available {}".format(torch.cuda.is_available()))
print("num gpus {}".format(torch.cuda.device_count()))

torch version 1.4.0
cuda available False
num gpus 0


In [4]:
# ImageFolder must contain subfolders of images:
image_dir = os.getcwd()+'/data'
print(image_dir)

/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/Code-Python/pytorch-snippets/data


In [5]:
image_transforms = transforms.Compose([
   transforms.Resize(224),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# load a dataset, create a loader
batch_size = 2
dataset = dsets.ImageFolder(root=image_dir,
                           transform=image_transforms)
loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
# load pre-trained alex-net
model = models.alexnet(pretrained=True)
if torch.cuda.is_available():
   # model = torch.nn.DataParallel(model, device_ids= range(torch.cuda.device_count()))
   model.cuda()  
# print the model, figure out which layers you want to get outputs from    
# print(model)

In [8]:
# connect forward hooks to desired layers
outputs = []
def hook(module, input, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
#     print('Inside ' + module.__class__.__name__ + ' forward')
#     print('')
#     print('input: ', type(input))
#     print('input[0]: ', type(input[0]))
#     print('output: ', type(output))
#     print('')
#     print('input size:', input[0].size())
#     print('output size:', output.data.size())
#     print('output norm:', output.data.norm())
    outputs.append(output)

#adding forward hook to layer conv2 (feature 3)
model.features[3].register_forward_hook(hook)

In [9]:
# grab a batch of images
images, labels = next(iter(loader))

In [10]:
# run the model on those images
if torch.cuda.is_available():
   images = Variable(images.cuda())
else:
   images = Variable(images)

In [11]:
out = model(images)
# inspect output
print(out)

tensor([[ 1.5445, -4.1327, -4.5404,  ...,  8.4978, 14.0044,  3.9530],
        [ 0.5043, -8.6503, -5.2454,  ...,  3.0968,  4.3191,  3.0388]],
       grad_fn=<AddmmBackward>)


In [12]:
# class for saving features from model
class SaveFeatures():
   features=None
   def __init__(self, m): 
        self.hook = m.register_forward_hook(self.hook_fn)
   def hook_fn(self, module, input, output): 
    self.features = output
   def remove(self): 
    self.hook.remove()

In [13]:
SaveFeatures(model.features[3]).remove()